Feature Engineering using Zensus Data

In [1]:
import pandas as pd
import geopandas as gpd
import folium 
import matplotlib.pyplot as plt

from pysal.lib import weights  
import segregation as seg

C:\Users\Bicakci\AppData\Local\Temp\ipykernel_17132\2746412016.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [3]:
zensus_df = gpd.read_file("../data/interim/merged_data_amenities.gpkg")

Segregation

Segregation based on age 

In [4]:
age_over_65 = "alk_65_over"; age_below_18 = "alk_18_under"; total_pop_var = "population_total_units"
    
zensus_df[total_pop_var] = zensus_df[total_pop_var].astype(int)
zensus_df[age_over_65] = zensus_df[age_over_65].astype(int) ; zensus_df[age_below_18] = zensus_df[age_below_18].astype(int)

# Filter out units with zero population
zensus_df = zensus_df[zensus_df[total_pop_var] > 0]
    
# A-spatial segregation index     
int_age_65 = seg.singlegroup.Interaction(data = zensus_df, group_pop_var = age_over_65, total_pop_var = total_pop_var)
int_age_18 = seg.singlegroup.Interaction(data = zensus_df, group_pop_var = age_below_18, total_pop_var = total_pop_var)

# Compute segregation indices for each neighborhood
int_age_65 = zensus_df.groupby(Neighborhood_Name).apply(lambda x: seg.singlegroup.Interaction(data=x, group_pop_var=age_over_65, total_pop_var=total_pop_var).statistic)
int_age_18 = zensus_df.groupby(neighborhood_var).apply(lambda x: seg.singlegroup.Interaction(data=x, group_pop_var=age_below_18, total_pop_var=total_pop_var).statistic)

print("Interaction of age category %s: %.2f and Interaction of age category %s: %.2f" %(age_over_65, int_age_65.statistic, age_below_18, int_age_18.statistic))

Interaction of age category alk_65_over: 0.74 and Interaction of age category alk_18_under: 0.79


In [31]:
# Define variables and data types
age_over_65 = "alk_65_over"
age_below_18 = "alk_18_under"
total_pop_var = "population_total_units"
single_var = "fam_single"
married_var = "fam_married"
german_nationality = "sk_germany"
other_nationality = "sk_abroad"
zensus_df[total_pop_var] = zensus_df[total_pop_var].astype(int)
zensus_df[age_over_65] = zensus_df[age_over_65].astype(int)
zensus_df[age_below_18] = zensus_df[age_below_18].astype(int)
zensus_df[single_var] = zensus_df[single_var].astype(int)
zensus_df[married_var] = zensus_df[married_var].astype(int)
zensus_df[german_nationality] = zensus_df[german_nationality].astype(int) 
zensus_df[other_nationality] = zensus_df[other_nationality].astype(int)


# Create an empty dictionary to store results for each neighborhood
segregation = {}

# Loop over each neighborhood and compute the segregation indices
for i, nb_name in enumerate(zensus_df.Neighborhood_Name):
    nb = zensus_df.loc[zensus_df.Neighborhood_Name == nb_name]
    
    # Filter the data to include only the current neighborhood
    nb_zensus = zensus_df[zensus_df.Neighborhood_Name == nb_name].copy()
    
    # Compute the segregation indices for the current neighborhood
    int_age_65 = seg.singlegroup.Interaction(data=nb_zensus, group_pop_var=age_over_65, total_pop_var=total_pop_var)
    int_age_18 = seg.singlegroup.Interaction(data=nb_zensus, group_pop_var=age_below_18, total_pop_var=total_pop_var)
    int_single = seg.singlegroup.Interaction(data=nb_zensus, group_pop_var=single_var, total_pop_var=total_pop_var)
    int_married = seg.singlegroup.Interaction(data=nb_zensus, group_pop_var=married_var, total_pop_var=total_pop_var)
    int_german_nationality = seg.singlegroup.Interaction(data=nb_zensus, group_pop_var=german_nationality, total_pop_var=total_pop_var)
    int_other_nationality = seg.singlegroup.Interaction(data=nb_zensus, group_pop_var=other_nationality, total_pop_var=total_pop_var)
    
    # Store the results for the current neighborhood
    segregation[nb_name] = {'int_age_65': int_age_65.statistic,
                             'int_age_18': int_age_18.statistic,
                             'int_single': int_single.statistic,
                             'int_married': int_married.statistic,
                             'int_german_nationality': int_german_nationality.statistic,
                             'int_other_nationality': int_other_nationality.statistic}




,int_age_65,int_age_18,int_single,int_married,int_german_nationality,int_other_nationality
Junkersdorf,0.744007,0.741246,0.506653,0.534465,0.073784,0.793515
Weiden,0.707986,0.815194,0.579252,0.539648,0.113418,0.800656
Müngersdorf,0.594723,0.775893,0.481525,0.514796,0.098158,0.684048
Lövenich,0.727496,0.773663,0.593238,0.489308,0.065499,0.866470
Widdersdorf,0.722837,0.706794,0.578754,0.477760,0.082422,0.831585
...,...,...,...,...,...,...
Seeberg,0.727174,0.766939,0.594434,0.494910,0.262461,0.636462
Merkenich,0.765966,0.761492,0.578313,0.479539,0.100393,0.723896
Fühlingen,0.758169,0.762218,0.579257,0.480795,0.075568,0.794135
Chorweiler,0.684908,0.748017,0.564479,0.535058,0.330202,0.599776


In [33]:
pd.merge(zensus_df,pd.DataFrame(segregation).transpose(),on=)

,Grid_Code,City_Code,buildings_total_units,n_occupied_by_owner,n_ownership_with_current_household,n_owned_without_current_household,n_rented_for_residential_purposes,n_rented_with_current_household,n_rented_without_current_household,n_vacation_and_leisure,...,animal_training,waste_basket;bicycle_parking,Nachhilfe,therapy,lounger,water,former school,adult_gaming_centre,geometry,re_christian
88,100mN30943E40993,05315000,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4099300.000 3094300.000, 4099300.000...",6
98,100mN30944E40992,05315000,3,0,0,0,0,3,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4099200.000 3094400.000, 4099200.000...",6
105,100mN30944E40999,05315000,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4099900.000 3094400.000, 4099900.000...",0
110,100mN30945E40994,05315000,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4099400.000 3094500.000, 4099400.000...",0
111,100mN30945E40995,05315000,6,0,0,0,0,4,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((4099500.000 3094500.000, 4099500.000...",10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45222,100mN31080E41025,05315000,9,0,9,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"POLYGON ((4102500.000 3108000.000, 4102500.000...",10
45223,100mN31080E41026,05315000,13,0,13,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"POLYGON ((4102600.000 3108000.000, 4102600.000...",15
45224,100mN31080E41027,05315000,16,0,16,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"POLYGON ((4102700.000 3108000.000, 4102700.000...",24
45225,100mN31080E41028,05315000,13,0,13,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"POLYGON ((4102800.000 3108000.000, 4102800.000...",19


Segregation based on being single or married

In [5]:
# Define variable names for single and married households
single_var = "fam_single"
married_var = "fam_married"

# Convert data types to integers
zensus_df[single_var] = zensus_df[single_var].astype(int)
zensus_df[married_var] = zensus_df[married_var].astype(int)

# Filter out units with zero population
#zensus_df = zensus_df[zensus_df[total_pop_var] > 0]

# Calculate segregation indices
int_single = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=single_var, total_pop_var=total_pop_var)
int_married = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=married_var, total_pop_var=total_pop_var)

# Print segregation indices
print("Interaction of %s households: %.2f" %(single_var, int_single.statistic))
print("Interaction of %s households: %.2f" %(married_var, int_married.statistic))


Interaction of fam_single households: 0.50
Interaction of fam_married households: 0.57


German nationality vs other nationality 

In [8]:
german_nationality = "sk_germany"
other_nationality = "sk_abroad"

zensus_df[german_nationality] = zensus_df[german_nationality].astype(int) 
zensus_df[other_nationality] = zensus_df[other_nationality].astype(int)

#zensus_df = zensus_df[zensus_df[total_pop_var] > 0]

int_german_nationality = seg.singlegroup.Interaction(data=zensus_df, 
                                                     group_pop_var=german_nationality, 
                                                     total_pop_var=total_pop_var)
int_other_nationality = seg.singlegroup.Interaction(data=zensus_df, 
                                                    group_pop_var=other_nationality, 
                                                    total_pop_var=total_pop_var)

print("Interaction of nationality category %s: %.2f and Interaction of nationality category %s: %.2f" 
      %(german_nationality, int_german_nationality.statistic, other_nationality, int_other_nationality.statistic))

Interaction of nationality category sk_germany: 0.14 and Interaction of nationality category sk_abroad: 0.73


Christian vs other 

In [23]:
re_catholic = zensus_df['re_roman_catholic']
re_protestant = zensus_df['re_protestant']
re_other = "re_other"

zensus_df[re_catholic] = zensus_df[re_catholic].astype(int)
zensus_df[re_protestant] = zensus_df[re_protestant].astype(int)
zensus_df[re_other] = zensus_df[re_other].astype(int)

#zensus_df = zensus_df[zensus_df[total_pop_var] > 0]

# A-spatial segregation index
int_catholic = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=re_catholic, total_pop_var=total_pop_var)
int_protestant = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=re_protestant, total_pop_var=total_pop_var)
int_other = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=re_other, total_pop_var=total_pop_var)

print("Interaction of Catholic religion: %.2f, Interaction of Protestant religion: %.2f and Interaction of other religions: %.2f" %(int_catholic.statistic, int_protestant.statistic, int_other.statistic))

KeyError: "None of [Index([ 6,  3,  0,  0, 10,  8,  8,  0,  0,  0,\n       ...\n       44, 50, 22,  7,  6,  5,  7, 16, 12, 30],\n      dtype='int32', length=14091)] are in the [columns]"

Built before 1919 vs after 2009

In [15]:
built_before_1919 = "j_before_1919"
built_2009_and_later = "j_2009_and_later"
total_building_var = "geb_total_units"

zensus_df[built_before_1919] = zensus_df[built_before_1919].astype(int)
zensus_df[built_2009_and_later] = zensus_df[built_2009_and_later].astype(int)

zensus_df = zensus_df[zensus_df[total_building_var] > 0]

int_built_before_1919 = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=built_before_1919, total_pop_var=total_building_var)
int_built_2009_and_later = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=built_2009_and_later, total_pop_var=total_building_var)

print("Interaction of construction year %s: %.2f and Interaction of construction year %s: %.2f" %(built_before_1919, int_built_before_1919.statistic, built_2009_and_later, int_built_2009_and_later.statistic))


KeyError: 'geb_total_units'

small vs big houses 

In [17]:
# Define variable names for apartment sizes
small_apt_var = "w_less_30"
large_apt_var = "w_100_and_more"

# Convert data types to integers
zensus_df[small_apt_var] = zensus_df[small_apt_var].astype(int)
zensus_df[large_apt_var] = zensus_df[large_apt_var].astype(int)

# Filter out units with zero population
#zensus_df = zensus_df[zensus_df[total_pop_var] > 0]

# Calculate segregation indices
int_small_apt = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=small_apt_var, total_pop_var=total_building_var)
int_large_apt = seg.singlegroup.Interaction(data=zensus_df, group_pop_var=large_apt_var, total_pop_var=total_building_var)

# Print segregation indices
print("Interaction of %s apartments: %.2f" %(small_apt_var, int_small_apt.statistic))
print("Interaction of %s apartments: %.2f" %(large_apt_var, int_large_apt.statistic))


ValueError: group_pop_var and total_pop_var columns must be present on the dataframe